# The QR Decomposition Exercise

### Suleyman Gozen
 
 I thank Yung-Hsu Tsui for his valuable comments.

In [1]:
import numpy as np
from scipy import linalg as la

In [2]:
""" Problem 1 """

def reduced_qr(A):
    m,n = A.shape
    Q = np.copy(A)
    R = np.zeros((n,n))
    
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] = Q[:,i]/R[i,i]
        for j in range(i+1, n):
            R[i,j] = Q[:,j].T@Q[:,i]
            Q[:,j] += -R[i,j]*Q[:,i]
    
    return Q,R
            
            
# Generate a random matrix and get its reduced QR decomposition via SciPy.
A = np.random.random((6,4))
Qprime,Rprime = la.qr(A, mode="economic") # Use mode="economic" for reduced QR.
Q, R = reduced_qr(A)
print(A.shape, Qprime.shape, Rprime.shape)
print(A.shape, Q.shape, R.shape)

# Verify that R is upper triangular, Q is orthonormal, and QR = A.
print(np.allclose(np.triu(Rprime), Rprime))
print(np.allclose(np.dot(Qprime.T, Qprime), np.identity(4)))
print(np.allclose(np.dot(Qprime, Rprime), A))

print(np.allclose(np.triu(R), R))
print(np.allclose(np.dot(Q.T, Q), np.identity(4)))
print(np.allclose(np.dot(Q, R), A))

(6, 4) (6, 4) (4, 4)
(6, 4) (6, 4) (4, 4)
True
True
True
True
True
True


In [3]:
""" Problem 2 """
def abs_det(A):
    det = np.product(np.diag(reduced_qr(A)[1]))
    return det

#compare scipy det and my det
A = np.random.random((4,4))
print(abs(la.det(A)))
print(abs_det(A))

0.032641055916978634
0.032641055917


In [4]:
""" Problem 3 """

def Axb(A,b):
    Q, R = reduced_qr(A)
    y = Q.T@b
    x = np.zeros_like(b)
    n = R.shape[0]
    
    x[-1] = y[-1]/ R[-1, -1] 
    
    for i in range(n-2, -1, -1):
        x[i] = y[i]     
        for j in range(i+1, n):
            x[i] += -x[j]* R[i,j]        
        x[i] = x[i]/R[i,i]
    
    return x

b = np.ones(4)
print(la.solve(A,b))
print(Axb(A,b))

[ 4.33089673 -3.60016664  5.13541337 -2.05435747]
[ 4.33089673 -3.60016664  5.13541337 -2.05435747]


In [5]:
""" Problem 4 """

def HH_QR(A):
    m,n = A.shape
    R = np.copy(A)
    Q = np.eye(m)

    for k in range(0,n):
        u = np.copy(R[k:,k])
        sign = lambda x: 1 if x >= 0 else -1
        u[0] += sign(u[0])*la.norm(u)
        u = u/la.norm(u)
        R[k:,k:] += - 2*np.outer(u,np.dot(u.T, R[k:,k:] ))
        Q[k:,:] += - 2*np.outer(u,np.dot(u.T, Q[k:,:] ))
        
    return Q.T, R

A = np.random.random((5, 3))
Qprime,Rprime = la.qr(A)
Q, R = HH_QR(A)
print (A.shape, Q.shape, R.shape)
print(np.allclose(Q.dot(R), A))
print (A.shape, Qprime.shape, Rprime.shape)
print(np.allclose(Qprime.dot(Rprime), A))


(5, 3) (5, 5) (5, 3)
True
(5, 3) (5, 5) (5, 3)
True


In [6]:
""" Problem 5 """

def hessenberg(A):
    m,n = A.shape
    H = np.copy(A)
    Q = np.eye(m)

    for k in range(n-2):
        u = np.copy(H[k+1:,k])
        sign = lambda x: 1 if x >= 0 else -1
        u[0] += sign(u[0])*la.norm(u)
        u = u/la.norm(u)
        H[k+1:,k:] -= 2*np.outer(u,np.dot(u.T, H[k+1:,k:]))
        H[:,k+1:] -= 2*np.outer(np.dot(H[:,k+1:],u),u.T)
        Q[k+1:,:] -= 2*np.outer(u,np.dot(u.T, Q[k+1:,:] ))       
        
    return H, Q.T

A = np.random.random((8,8))
# Verify that H has all zeros below the first subdiagonal and QHQ^T = A.
Hprime, Qprime = la.hessenberg(A, calc_q=True)
print(np.allclose(np.triu(Hprime, -1), Hprime))
print(np.allclose(np.dot(np.dot(Qprime, Hprime), Qprime.T), A))

H, Q = hessenberg(A)
print(np.allclose(np.triu(H, -1), H))
print(np.allclose(np.dot(np.dot(Q, H), Q.T), A))

True
True
True
True
